In [ ]:
!ls

sample_data


#External Databases

In [ ]:
#! pip install git+https://github.com/huggingface/transformers.git
!pip install transformers==4.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 45.1 MB/s eta 0:00:00


In [ ]:
!pip install "ray[tune]" torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.7 MB/s eta 0:00:00


In [ ]:
!pip install torch-tensorrt #==1.1.0 --find-links https://github.com/pytorch/TensorRT/releases/expanded_assets/v1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 65.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 839.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=8f3bd68cb0f400231b22873bc3cf8ea5e843c58e0d49db5f7d6079820dace6b6
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
#pip install accelerate -U

#Libraries

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

from google.colab import files

import os
import re
import json
import string

import torch
import torch.nn as nn

import numpy as np
#import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers

from datasets import load_metric
from sklearn.metrics import f1_score, precision_score, recall_score
from seqeval.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as prfs

import tensorflow_hub as hub
from keras import backend as K

import transformers
print(transformers.__version__)
from transformers import TrainingArguments
from transformers import Trainer
from transformers.trainer_utils import EvalLoopOutput

from transformers import AutoTokenizer, AutoModel
BertTokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
BertEmbModel = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')



4.20.0


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Loading Dataset NCBI

In [ ]:
class NCBIDataset(torch.utils.data.Dataset):
  def __init__(self, raw_dataset, max_length=250):
    self.raw_x = [x['tokens'] for x in raw_dataset]
    self.raw_y = [x['ner_tags'] for x in raw_dataset]

    self.max_length = max_length

  def tokenize_and_preserve_labels(self, sentence, text_labels):
    """
    The tokenizer can split single words into multiple tokens - this breaks
    the labels, so we need to keep track of this!
    """
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = BertTokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

  def __getitem__(self, idx):
    tokens = self.raw_x[idx]
    labels = np.zeros((len(tokens)))
    for i, label in enumerate(self.raw_y[idx]):
      labels[i] = label

    # This could be moved to __init__ to save time?
    tokens, labels = self.tokenize_and_preserve_labels(tokens, labels)

    # Convert each token to an id number
    input_ids = BertTokenizer.convert_tokens_to_ids(tokens)

    # add and adjust for special tokens
    input_ids = [BertTokenizer.cls_token_id] + input_ids + [BertTokenizer.sep_token_id]
    labels = [0] + labels + [0]

    # Pad inputs
    input_ids = torch.tensor(np.pad(input_ids, [0, self.max_length-len(input_ids)]))
    labels = torch.tensor(np.pad(labels, [0, self.max_length-len(labels)], constant_values=-100))

    for x in range(len(labels)):
      if labels[x] == 1:
        labels[x] = 9
      if labels[x] == 2:
        labels[x] = 10

    attention_mask = torch.tensor([int(i != 0) for i in input_ids])

    #return {input_ids, labels}
    #return {'input_ids': [input_ids], 'labels': labels}
    return {'input_ids': [input_ids], 'attention_mask': [attention_mask], 'labels': labels}

  def __len__(self):
    return len(self.raw_y)

In [ ]:
# Entity mapping, uses BIO tags
def Entity2ID(Sc_Entity):
  return {
        'B': 9,
        'I': 10,
        'O': 0,
    }[Sc_Entity]

def ID2Entity(Sc_Entity):
  return {
        9: 'B-OtherScientificTerm',
        10: 'I-OtherScientificTerm',
        0: 'None',
    }[Sc_Entity]


In [ ]:
# Download dataset using datasets, might need to install it!
from datasets import load_dataset

raw_train = load_dataset('ncbi_disease', split='train')
raw_test = load_dataset('ncbi_disease', split='test')
raw_val = load_dataset('ncbi_disease', split='validation')

# Convert to torch datasets
train_NCBI = NCBIDataset(raw_train)
test_NCBI = NCBIDataset(raw_test)
val_NCBI = NCBIDataset(raw_val)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
len(val_NCBI)

924

In [ ]:
local_hp_train_NCBI = []
local_hp_val_NCBI = []
for x in range(500):
  local_hp_train_NCBI.append(train_NCBI[x])

for x in range(180):
  local_hp_val_NCBI.append(val_NCBI[x])

print(len(local_hp_train_NCBI))
print(local_hp_train_NCBI[1])

500
{'input_ids': [tensor([  102,   111, 24243,   861,   153,  7503,  1070,  5703,   145, 16036,
          546,  7421,   579, 14957,   787,  3151,   111, 12157, 10636,  3430,
          214,  8437,   106,  1127,   190, 18260, 12242,  4655,   239, 12186,
          145, 20362,   579,   239, 12186,   546,   422, 10361, 30111,  1352,
         6030,   107,   137,  6130, 14025,   205,   103,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   

In [ ]:
#train_NCBI[20]

#Loading Dataset SciTDM

In [ ]:
uploaded = files.upload()

Saving test_500_v2.conll to test_500_v2.conll
Saving train_1500_v2.conll to train_1500_v2.conll


In [ ]:
class TDMSciDataset(torch.utils.data.Dataset):
  def __init__(self, raw_x, raw_y, max_length=250):
    self.raw_x = raw_x
    self.raw_y = raw_y

    self.max_length = max_length

  def tokenize_and_preserve_labels(self, sentence, text_labels):
    """
    The tokenizer can split single words into multiple tokens - this breaks
    the labels, so we need to keep track of this!
    """
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = BertTokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

  def __getitem__(self, idx):
    tokens = self.raw_x[idx]
    labels = np.zeros((len(tokens)))
    for i, label in enumerate(self.raw_y[idx]):
      labels[i] = label

    # This could be moved to __init__ to save time?
    tokens, labels = self.tokenize_and_preserve_labels(tokens, labels)

    # Convert each token to an id number
    input_ids = BertTokenizer.convert_tokens_to_ids(tokens)

    # add and adjust for special tokens
    input_ids = [BertTokenizer.cls_token_id] + input_ids + [BertTokenizer.sep_token_id]
    labels = [0] + labels + [0]

    # Pad inputs
    input_ids = torch.tensor(np.pad(input_ids, [0, self.max_length-len(input_ids)]))
    labels = torch.tensor(np.pad(labels, [0, self.max_length-len(labels)], constant_values=-100))

    attention_mask = torch.tensor([int(i != 0) for i in input_ids])

    #return {input_ids, labels}
    #return {'input_ids': [input_ids], 'labels': labels}
    return {'input_ids': [input_ids], 'attention_mask': [attention_mask], 'labels': labels}

  def __len__(self):
    return len(self.raw_y)


In [ ]:
# Entity maps
def Entity2ID(Sc_Entity):
  return {
        'B-TASK': 1,
        'I-TASK': 2,
        'B-DATASET': 7,
        'I-DATASET': 8,
        'B-METRIC': 5,
        'I-METRIC': 6,
        'None': 0,
        'O': 0,
    }[Sc_Entity]

def ID2Entity(Sc_Entity):
  return {
        1: 'B-Task',
        2: 'I-Task',
        7: 'B-Material',
        8: 'I-Material',
        5: 'B-Metric',
        6: 'I-Metric',
        0: 'None',
    }[Sc_Entity]

In [ ]:
# Load raw data train
raw_x = []
raw_y = []

cur_x = []
cur_y = []
with open('train_1500_v2.conll', 'r') as f:
  for line in f:
    if len(line.strip()) <= 3:
      raw_x.append(cur_x)
      raw_y.append(cur_y)
      cur_x = []
      cur_y = []
    else:
      line = line.strip().split('\t')
      cur_x.append(line[0])
      cur_y.append(Entity2ID(line[-1]))  # use this for BIO tags
      # cur_y.append(Entity2ID(line[-1].split('-')[-1]))  # use this for span tags

print(raw_x[10])
print(raw_y[10])

# Create actual dataset
train_SciTDM = TDMSciDataset(raw_x, raw_y)

#print(train[0])

['The', 'results', 'show', 'that', 'proposed', 'shallow', 'representations', 'of', 'sentence', 'structure', 'are', 'robust', 'to', 'reductions', 'in', 'parsing', 'accuracy', ',', 'and', 'that', 'the', 'contribution', 'of', 'alternative', 'representations', 'of', 'sentence', 'structure', 'to', 'successful', 'semantic', 'role', 'labeling', 'varies', 'with', 'the', 'integrity', 'of', 'the', 'parsing', 'and', 'argument', '-', 'identification', 'stages', '.', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0]


In [ ]:
# Load raw data test
raw_x = []
raw_x = []
raw_y = []

cur_x = []
cur_y = []
with open('test_500_v2.conll', 'r') as f:
  for line in f:
    if len(line.strip()) <= 3:
      raw_x.append(cur_x)
      raw_y.append(cur_y)
      cur_x = []
      cur_y = []
    else:
      line = line.strip().split('\t')
      cur_x.append(line[0])
      cur_y.append(Entity2ID(line[-1]))  # use this for BIO tags
      # cur_y.append(Entity2ID(line[-1].split('-')[-1]))  # use this for span tags

print(raw_x[10])
print(raw_y[10])

# Create actual dataset
test_SciTDM = TDMSciDataset(raw_x, raw_y)

#print(test[0])

['We', 'evaluate', 'our', 'approach', 'on', 'three', 'datasets', ':', 'TriviaQA', 'unfiltered', '(', ',', 'a', 'dataset', 'of', 'questions', 'from', 'trivia', 'databases', 'paired', 'with', 'documents', 'found', 'by', 'completing', 'a', 'web', 'search', 'of', 'the', 'questions', ';', 'TriviaQA', 'web', ',', 'a', 'dataset', 'derived', 'from', 'TriviaQA', 'unfiltered', 'by', 'treating', 'each', 'question', 'document', 'pair', 'where', 'the', 'document', 'contains', 'the', 'question', 'answer', 'as', 'an', 'individual', 'training', 'point', ';', 'and', 'SQuAD', '(', ',', 'a', 'collection', 'of', 'Wikipedia', 'articles', 'and', 'crowdsourced', 'questions', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(test_SciTDM), len(train_SciTDM)

(486, 1522)

In [ ]:
#train[1]

In [ ]:
local_hp_train_SciTDM = []
local_hp_val_SciTDM = []
for x in range(500):
  local_hp_train_SciTDM.append(train_SciTDM[x])

for x in range(180):
  local_hp_val_SciTDM.append(test_SciTDM[x])

print(len(local_hp_train_SciTDM))
print(local_hp_train_SciTDM[1])

500
{'input_ids': [tensor([ 102,  185, 3138,  111, 8709, 1910,  214, 5373,  547, 3838, 2683,  168,
        9810, 8158,  121, 1323, 4371,  205,  103,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  

#Loading Dataset SciERC

In [ ]:
uploaded = files.upload()

Saving dev.json to dev.json
Saving test.json to test.json
Saving train.json to train.json


In [ ]:
class SciERCDataset(torch.utils.data.Dataset):
  def __init__(self, raw_data, entity_map, max_length=250):
    # Need to fix sentence offset for labels!!
    for i, sample in enumerate(raw_data):
      sent_offset = 0
      for j, sent in enumerate(sample['sentences']):
        for k, ner in enumerate(sample['ner'][j]):
          raw_data[i]['ner'][j][k] = [raw_data[i]['ner'][j][k][0] - sent_offset,
                                      raw_data[i]['ner'][j][k][1] - sent_offset,
                                      raw_data[i]['ner'][j][k][2]]
        sent_offset += len(sent)

    sentences = [x['sentences'] for x in raw_data]
    self.raw_x = [sent for sublist in sentences for sent in sublist]

    ners = [x['ner'] for x in raw_data]
    self.raw_y = [ner for sublist in ners for ner in sublist]

    self.entity_map = entity_map
    self.max_length = max_length

  def tokenize_and_preserve_labels(self, sentence, text_labels):
    """
    The tokenizer can split single words into multiple tokens - this breaks
    the labels, so we need to keep track of this!
    """
    tokenized_sentence = []
    labels = []
    prev_label = 0

    for word, label in zip(sentence, text_labels):
        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = BertTokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        if label != 0:
          labels.extend([-100] * n_subwords)
          labels[-1 * n_subwords] = label
        else:
          labels.extend([label] * n_subwords)

        prev_label = label

    return tokenized_sentence, labels

  def __getitem__(self, idx):
    tokens = self.raw_x[idx]
    labels = np.zeros((len(tokens)))

    #For Span
    for ner in self.raw_y[idx]:  # Returns [start_idx, end_idx, string label]
      assert len(tokens) >= ner[0], '{}, {}'.format(tokens, ner)
      ner_name = 'I-' + ner[-1]
      label = self.entity_map(ner_name)  # Get the integer label
      labels[ner[0]:ner[1]+1] = label
      labels[ner[0]] = label - 1

    #For BIO
    #for ner in self.raw_y[idx]:  # Returns [start_idx, end_idx, string label]
    #  assert len(tokens) >= ner[0], '{}, {}'.format(tokens, ner)  # Double checking the labels are correct
    #  ner_name = 'I-' + ner[-1]  # assume all are "I" labels
    #  label = self.entity_map(ner_name)  # Get the integer label
    #  labels[ner[0]:ner[1]+1] = label
    #  labels[ner[0]] = label - 1  # Change first one to "B" label

    # This could be moved to __init__ to save time?
    tokens, labels = self.tokenize_and_preserve_labels(tokens, labels)

    # Convert each token to an id number
    input_ids = BertTokenizer.convert_tokens_to_ids(tokens)

    # add and adjust for special tokens
    input_ids = [BertTokenizer.cls_token_id] + input_ids + [BertTokenizer.sep_token_id]
    labels = [0] + labels + [0]

    # Pad inputs
    input_ids = torch.tensor(np.pad(input_ids, [0, self.max_length-len(input_ids)]))
    labels = torch.tensor(np.pad(labels, [0, self.max_length-len(labels)], constant_values=-100))

    attention_mask = torch.tensor([int(i != 0) for i in input_ids])

    return {'input_ids': [input_ids], 'attention_mask': [attention_mask], 'labels': labels}

  def __len__(self):
    return len(self.raw_y)

In [ ]:
def Entity2ID(Sc_Entity):
  return {
        'B-Task': 1,
        'I-Task': 2,
        'B-Method': 3,
        'I-Method': 4,
        'B-Metric': 5,
        'I-Metric': 6,
        'B-Material': 7,
        'I-Material': 8,
        'B-OtherScientificTerm': 9,
        'I-OtherScientificTerm': 10,
        'B-Generic': 11,
        'I-Generic': 12,
        'None': 0
    }[Sc_Entity]

def ID2Entity(Sc_Entity):
  return {
        1: 'B-Task',
        2: 'I-Task',
        3: 'B-Method',
        4: 'I-Method',
        5: 'B-Metric',
        6: 'I-Metric',
        7: 'B-Material',
        8: 'I-Material',
        9: 'B-OtherScientificTerm',
        10: 'I-OtherScientificTerm',
        11: 'B-Generic',
        12: 'I-Generic',
        0: 'O'
    }[Sc_Entity]

In [ ]:
train_data = []
with open('train.json','r') as f:
  for line in f:
    train_data.append(json.loads(line))

test_data = []
with open('test.json','r') as f:
  for line in f:
    test_data.append(json.loads(line))

val_data = []
with open('dev.json','r') as f:
  for line in f:
    val_data.append(json.loads(line))

#train_data[3]['sentences'][0], train_data[3]['ner'][0], len(train_data[3]['sentences'][1])
len(train_data)

350

In [ ]:
train_SciERC = SciERCDataset(train_data, Entity2ID)
val_SciERC = SciERCDataset(val_data, Entity2ID)
test_SciERC = SciERCDataset(test_data, Entity2ID)

In [ ]:
#tokens = BertTokenizer.convert_ids_to_tokens(train[1]['input_ids'][0])
#labels = train[1]['labels']
#print(' '.join(tokens))
#print(labels)

In [ ]:
train_SciERC[0]

{'input_ids': [tensor([  102,  6170,   165,   817,   147,   195,   428,   579,  2220,   579,
           2159,   191,   111,  2525,   131,  7395, 30113,   131,   111,  1222,
           1211,   198,  3862,  8503, 13510,  2057,   579,  8595,  4111,   205,
            103,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

#Unifiying Loaded Datasets

In [ ]:
len(train_NCBI), len(train_SciTDM), len(train_SciERC)

(5433, 1522, 1861)

In [ ]:
len(test_NCBI), len(test_SciTDM), len(test_SciERC)

(941, 486, 551)

In [ ]:
len(val_NCBI), len(val_SciERC)

(924, 275)

In [ ]:
len(local_hp_train_NCBI), len(local_hp_train_SciTDM), len(local_hp_val_NCBI), len(local_hp_val_SciTDM)

(500, 500, 180, 180)

In [ ]:
train = train_NCBI + train_SciTDM + train_SciERC
test = test_NCBI + test_SciTDM + test_SciERC
val = val_NCBI + val_SciERC + local_hp_val_SciTDM
local_hp_train = local_hp_train_NCBI + local_hp_train_SciTDM
local_hp_val = local_hp_val_NCBI + local_hp_val_SciTDM
print(len(train), len(test), len(val), len(local_hp_train), len(local_hp_val))
print(train[2000])

8816 1978 1379 1000 360
{'input_ids': [tensor([  102,   106,   983,   422, 15162, 30118, 30130,   422,  6525,   121,
          106,  1454,   190,  5390, 30113,  5855,   422,   434,   528,  1887,
          137,   797,   147, 12886,   106,   787,   190, 11600,   147, 16745,
         1352, 10837, 29730,  7948,  4446,  1391,   145, 16745,  1352, 10837,
        15325,   546,   205,   103,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    

# Pytorch Model Definition

In [ ]:
class NerModel(nn.Module):
  def __init__(self, b_embeddings, emb_dims=768, ff_dims=14, out_dims=13):
    super(NerModel, self).__init__()
    self.sci_embeddings = b_embeddings
    self.embd_dropout = nn.Dropout(0.1)
    self.ff_dropout = nn.Dropout(0.1)
    self.ff = nn.Linear(emb_dims, ff_dims)
    #self.tanh = nn.Tanh()#
    #self.lstm = nn.LSTM(768, 100, 1, bidirectional=True)#
    #self.lstm_drop = nn.Dropout(0.4)#
    self.ff = nn.Linear(768, 14)#200
    self.ff_act = nn.ReLU()
    self.classifier = nn.Linear(ff_dims, out_dims)
  def forward(self, input_ids=None, attention_mask=None, labels=None):
    embds = self.sci_embeddings(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
    out = self.embd_dropout(embds)
    #out, _ = self.lstm(out)#
    #out = self.tanh(out)#
    #out = self.lstm_drop(out)#
    out = self.ff(out)
    out = self.ff_act(out)
    out = self.ff_dropout(out)
    out = self.classifier(out)
    return out

# Metrics and Configs

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    #print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

import math

def load_param():
  for n, v in best_run.hyperparameters.items():
    if n == 'seed':
      setattr(trainer.args, n, math.ceil(v))
      print(n, math.ceil(v))
    else:
      setattr(trainer.args, n, v)
      print(n, v)

In [ ]:
batch_size = 4
training_args = TrainingArguments(
    "trained_scibert_ner_model", # output dir
    learning_rate=1e-5,
    num_train_epochs=10,
    dataloader_drop_last=True,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    logging_steps=50,
    save_steps=len(train) // batch_size,
    lr_scheduler_type='cosine',
    evaluation_strategy='steps',
    eval_steps=len(train) // batch_size)
#print(training_args)

In [ ]:
def collator(batch):
  out =  {
      'input_ids': torch.stack([(x['input_ids'][0]) for x in batch]),
      'attention_mask': torch.stack([x['attention_mask'][0] for x in batch]),
      'labels': torch.stack([x['labels'].clone().detach() for x in batch])
  }
  return out

In [ ]:
import torch.nn.functional as F

class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight, gamma, reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, ignore_index=-100, reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [ ]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_labels=13):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs

        #weights = torch.tensor([0.1, 1.5, 1.55, 1.9, 1.75, 1.65, 1.9]).cuda()  # The no-class label has too many examples, we need to weight the loss - this probably needs further tuning

        #weights = torch.tensor([0.1, 1.5, 1.2, 1.9, 1.75, 1.3, 1.85]).cuda()

        #weights = torch.tensor([0.89, 1.73, 1.69, 1.79, 1.76, 1.7, 1.77]).cuda()

        #weights = torch.tensor([0.89, 1.69, 1.65, 1.84, 1.75, 1.66, 1.77]).cuda()

        #weights = torch.tensor([0.95, 1.6, 1.5, 1.94, 1.7, 1.55, 1.83]).cuda()

        #weights = torch.tensor([0.12, 0.83, 0.78, 1.0, 0.88, 0.8, 0.95]).cuda()

        #weights = torch.tensor([0.42, 1.5, 1.2, 2.0, 1.6, 1.4, 1.8]).cuda()

        #weights = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]).cuda()

        #weights = torch.tensor([0.1, 1.36, 0.93, 7.45, 2.66, 1.01, 3.51]).cuda()

        #weights = torch.tensor([0.26, 0.95, 0.92, 0.99, 0.97, 0.93, 0.98]).cuda()

        weights = torch.tensor([0.25, 0.98, 0.97, 0.97, 0.95, 1.0, 1.0, 0.99, 0.98, 0.97, 0.96, 0.98, 1.2]).cuda()

        gamma=5
        loss_fct = FocalLoss(weight=weights, gamma=gamma)
        loss = loss_fct(logits.view(-1, num_labels), labels.long().view(-1))
        return (loss, outputs) if return_outputs else loss

    def evaluation_loop(self, dataloader, description, prediction_loss_only=None, ignore_keys=None, metric_key_prefix="eval", num_labels=7):
      args = self.args
      prediction_loss_only = prediction_loss_only if prediction_loss_only is not None else args.prediction_loss_only

      self.model.eval()

      all_losses = []
      all_preds = []
      all_labels = []
      for step, sample in enumerate(dataloader):
        for i in range(0, len(sample['labels'])):
          inputs = {}
          inputs['input_ids'] = torch.stack([sample['input_ids'][i].cuda()])
          inputs['attention_mask'] = torch.stack([sample['attention_mask'][i].cuda()])
          inputs['labels'] = torch.stack([sample['labels'][i].cuda()])
          labels = inputs['labels'][0].cpu().numpy()

          (loss, logits) = self.compute_loss(self.model, inputs, return_outputs=True)
          logits = logits[0].cpu().detach().numpy()
          preds = np.argmax(nn.Softmax(dim=-1)(torch.tensor(logits)).numpy(), axis=-1)

          all_losses = np.concatenate((all_losses, [loss.detach().cpu().numpy()]), axis=0)

          preds = preds[labels != -100]
          labels = labels[labels != -100]
          all_preds = np.concatenate((all_preds, preds))
          all_labels = np.concatenate((all_labels, labels))

      metrics = {}
      metrics['macro_f1'] = f1_score(all_labels, all_preds, average='macro')
      metrics['macro_precision'] = precision_score(all_labels, all_preds, average='macro', zero_division=0)
      metrics['macro_recall'] = recall_score(all_labels, all_preds, average='macro', zero_division=0)
      metrics['micro_f1'] = f1_score(all_labels, all_preds, average='micro')
      metrics['micro_precision'] = precision_score(all_labels, all_preds, average='micro', zero_division=0)
      metrics['micro_recall'] = recall_score(all_labels, all_preds, average='micro', zero_division=0)

      metrics['macro_f1_no_o'] = f1_score(all_labels, all_preds, average='macro', labels=[1, 2, 3, 4, 5, 6])
      metrics['macro_precision_no_o'] = precision_score(all_labels, all_preds, average='macro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)
      metrics['macro_recall_no_o'] = recall_score(all_labels, all_preds, average='macro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)
      metrics['micro_f1_no_o'] = f1_score(all_labels, all_preds, average='micro', labels=[1, 2, 3, 4, 5, 6])
      metrics['micro_precision_no_o'] = precision_score(all_labels, all_preds, average='micro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)
      metrics['micro_recall_no_o'] = recall_score(all_labels, all_preds, average='micro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)

      for key in list(metrics.keys()):
        if not key.startswith(metric_key_prefix):
          metrics[metric_key_prefix + '_' + key] = metrics.pop(key)

      metrics[metric_key_prefix + '_loss'] = all_losses.mean().item()

      return EvalLoopOutput(predictions=all_preds, label_ids=all_labels, metrics=metrics, num_samples=len(dataloader))

In [ ]:
def my_hp_space_ray(trial):
    from ray import tune

    return {
        "learning_rate": tune.loguniform(9e-6, 1e-4),
        "num_train_epochs": tune.choice(range(5, 25)),
        "weight_decay": tune.uniform(0.0, 0.2),
        "per_device_train_batch_size": tune.choice([4, 8, 16]),  #<16 definetly not working
    }

In [ ]:
#train[0]

# Hyperparameter Tuning

In [ ]:
def model_init():
    x = NerModel(BertEmbModel)
    x.sci_embeddings.requires_grad = False
    return x

trainer = MultilabelTrainer(model_init=model_init,
                            args=training_args,
                            train_dataset=train,
                            eval_dataset=val,
                            data_collator=collator  # defines how to merge data into batches, using the collator function above
                            )

In [ ]:
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

best_run = trainer.hyperparameter_search(backend="ray",
                                         resources_per_trial={"gpu": 1, "cpu": 0},
                                         n_trials=18,
                                         direction="maximize",
                                         hp_space=my_hp_space_ray,
                                         search_alg=HyperOptSearch(metric='eval_micro_recall_no_o', mode="max"),  #'eval_*f1_micro'
                                         scheduler=ASHAScheduler(metric='eval_micro_recall_no_o', mode="max")
                                         )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
best_run

BestRun(run_id='6753986b', objective=8.574123831086196, hyperparameters={'learning_rate': 2.768901750544875e-05, 'num_train_epochs': 8, 'weight_decay': 0.0273138487143614, 'per_device_train_batch_size': 4})

# Cossvalidation

In [ ]:
#import tensorflow as tf
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from torch import nn
from transformers import Trainer

#print('GPU detected:', tf.config.list_physical_devices('GPU'))
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)
results = np.zeros(5)
resultss = np.zeros(5)
dataset = ConcatDataset([train, test])



In [ ]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    #train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    #test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    local_train = []
    i = 0
    for idx in train_ids:
      #if i <= 1920:
      local_train.append(dataset[idx])
      #i += 1

    local_test = []
    i = 0
    for idx in test_ids:
      #if i <= 480:
      local_test.append(dataset[idx])
      #i += 1

    training_args = TrainingArguments("trained_scibert_ner_model", # output dir
                                      learning_rate=2.768901750544875e-05,
                                      weight_decay=0.0273138487143614,
                                      num_train_epochs=8,
                                      dataloader_drop_last=True,
                                      per_device_eval_batch_size=4,
                                      per_device_train_batch_size=4,
                                      logging_steps=50,
                                      save_steps=len(local_train) // batch_size,
                                      lr_scheduler_type='cosine',
                                      evaluation_strategy='steps',
                                      eval_steps=len(local_train) // batch_size
                                      )

    #learning_rate 1.5441110254159498e-05
    #num_train_epochs 12
    #weight_decay 0.21467682833420043
    #per_device_train_batch_size 16


    # Init the neural network
    ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training
    ner_model.sci_embeddings.requires_grad = False

    trainer = MultilabelTrainer(model=ner_model,
                                args=training_args,
                                train_dataset=local_train,
                                eval_dataset=local_test,
                                data_collator=collator  # defines how to merge data into batches, using the collator function above
                                )

    #Loading Best parameters
    #load_param()

    #Training
    trainer.train()

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(ner_model.state_dict(), save_path)

    # Evaluationfor this fold
    #correct, total = 0, 0
    with torch.no_grad():
      result = trainer.evaluate(local_test)
      print(result)

      # Print accuracy
      print('Accuracy for fold ', fold, ': ', result['eval_micro_f1_no_o'], ' -- ', result['eval_micro_f1'])
      print('--------------------------------')
      results[fold] = result['eval_micro_f1_no_o']
      resultss[fold] = result['eval_micro_f1']
      del result



FOLD 0
--------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8635
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 17264


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
2158,0.021000,0.048026,0.591558,0.526223,0.697114,0.910806,0.910806,0.910806,0.647785,0.548724,0.794764,0.638010,0.533251,0.793991
4316,0.004800,0.054148,0.613942,0.532669,0.754255,0.922837,0.922837,0.922837,0.637092,0.526964,0.813532,0.637986,0.530488,0.800123
6474,0.001300,0.058260,0.648107,0.584845,0.740460,0.931833,0.931833,0.931833,0.681218,0.594595,0.808510,0.672121,0.576409,0.805947
8632,0.000500,0.057363,0.633148,0.551276,0.757556,0.924383,0.924383,0.924383,0.673833,0.572419,0.824106,0.668635,0.575215,0.798283
10790,0.000100,0.065241,0.675597,0.618340,0.754879,0.939819,0.939819,0.939819,0.726075,0.648785,0.826741,0.707467,0.626003,0.813305
12948,0.000200,0.061650,0.671331,0.612664,0.752944,0.938332,0.938332,0.938332,0.710284,0.633775,0.818980,0.693427,0.601668,0.818210
15106,0.000100,0.085389,0.682484,0.634577,0.743883,0.945172,0.945172,0.945172,0.732688,0.671449,0.808696,0.721918,0.665288,0.789086
17264,0.000000,0.085574,0.683822,0.634926,0.746845,0.944979,0.944979,0.944979,0.734107,0.668296,0.816928,0.721684,0.658081,0.798896


Saving model checkpoint to trained_scibert_ner_model/checkpoint-2158
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-4316
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-6474
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-8632
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-10790
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-12948
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-15106
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint 

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.68382229735842, 'eval_macro_precision': 0.6349262364698003, 'eval_macro_recall': 0.7468453400229537, 'eval_micro_f1': 0.9449789581691377, 'eval_micro_precision': 0.9449789581691377, 'eval_micro_recall': 0.9449789581691377, 'eval_macro_f1_no_o': 0.7341074407427798, 'eval_macro_precision_no_o': 0.6682963757547401, 'eval_macro_recall_no_o': 0.8169277556177517, 'eval_micro_f1_no_o': 0.7216837441152034, 'eval_micro_precision_no_o': 0.658080808080808, 'eval_micro_recall_no_o': 0.7988963825873697, 'eval_loss': 0.08557401420448488, 'eval_runtime': 38.1804, 'eval_samples_per_second': 14.117, 'eval_steps_per_second': 3.536, 'epoch': 8.0}
Accuracy for fold  0 :  0.7216837441152034  --  0.9449789581691377
--------------------------------
FOLD 1
--------------------------------


...

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.9270876710376388, 'eval_macro_precision': 0.8852010120508429, 'eval_macro_recall': 0.9751601754360164, 'eval_micro_f1': 0.985006372291776, 'eval_micro_precision': 0.985006372291776, 'eval_micro_recall': 0.985006372291776, 'eval_macro_f1_no_o': 0.926855376855895, 'eval_macro_precision_no_o': 0.8780704426137423, 'eval_macro_recall_no_o': 0.9822799509922172, 'eval_micro_f1_no_o': 0.9190482894551598, 'eval_micro_precision_no_o': 0.8650940895838855, 'eval_micro_recall_no_o': 0.9801801801801802, 'eval_loss': 0.0008043160018255715, 'eval_runtime': 38.2636, 'eval_samples_per_second': 14.087, 'eval_steps_per_second': 3.528, 'epoch': 8.0}
Accuracy for fold  3 :  0.9190482894551598  --  0.985006372291776
--------------------------------
FOLD 4
--------------------------------


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8636
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 17272


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
2159,0.000400,0.000896,0.907424,0.859130,0.973694,0.983011,0.983011,0.983011,0.928092,0.897422,0.965483,0.916303,0.864072,0.975255
4318,0.000200,0.000847,0.910348,0.869868,0.957323,0.982341,0.982341,0.982341,0.917628,0.873167,0.969366,0.904941,0.849011,0.968760
6477,0.000100,0.000525,0.917922,0.876368,0.968853,0.985632,0.985632,0.985632,0.941752,0.917937,0.967662,0.941740,0.912892,0.972471
8636,0.000000,0.001125,0.923948,0.878928,0.979504,0.986135,0.986135,0.986135,0.938903,0.904084,0.977633,0.937361,0.899148,0.978967
10795,0.000000,0.002436,0.927682,0.910104,0.949340,0.987019,0.987019,0.987019,0.934915,0.915150,0.959496,0.930983,0.893151,0.972162
12954,0.000000,0.001333,0.928496,0.889431,0.976013,0.987323,0.987323,0.987323,0.947942,0.922631,0.975813,0.946095,0.914550,0.979895
15113,0.000000,0.000666,0.935669,0.901066,0.976305,0.988512,0.988512,0.988512,0.948950,0.923978,0.976206,0.945938,0.914525,0.979586
17272,0.000000,0.000706,0.934667,0.898837,0.977089,0.988664,0.988664,0.988664,0.950070,0.924929,0.977415,0.947825,0.917245,0.980513


Saving model checkpoint to trained_scibert_ner_model/checkpoint-2159
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-4318
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-6477
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-8636
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-10795
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-12954
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-15113
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint 

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.9346671724293613, 'eval_macro_precision': 0.8988368779239028, 'eval_macro_recall': 0.9770894607265225, 'eval_micro_f1': 0.9886640663091175, 'eval_micro_precision': 0.9886640663091175, 'eval_micro_recall': 0.9886640663091175, 'eval_macro_f1_no_o': 0.9500701375534556, 'eval_macro_precision_no_o': 0.9249287157628564, 'eval_macro_recall_no_o': 0.9774146467304248, 'eval_micro_f1_no_o': 0.9478247869636717, 'eval_micro_precision_no_o': 0.9172453703703703, 'eval_micro_recall_no_o': 0.9805134549953604, 'eval_loss': 0.000706205445108781, 'eval_runtime': 38.391, 'eval_samples_per_second': 14.04, 'eval_steps_per_second': 3.516, 'epoch': 8.0}
Accuracy for fold  4 :  0.9478247869636717  --  0.9886640663091175
--------------------------------


# Results

In [ ]:
results

array([0.72168374, 0.86850998, 0.91612185, 0.91904829, 0.94782479])

In [ ]:
resultss

array([0.94497896, 0.97703046, 0.98314021, 0.98500637, 0.98866407])

In [ ]:
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
key = 0
for value in results:
  print(f'Fold {key}: {value} %')
  sum += value
  key += 1
print(f'Average: {sum/len(results)} %')

K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 0.7216837441152034 %
Fold 1: 0.8685099846390169 %
Fold 2: 0.9161218528307136 %
Fold 3: 0.9190482894551598 %
Fold 4: 0.9478247869636717 %
Average: 0.874637731600753 %


# Pytorch Training - Loop UPDT

In [ ]:
device = 'cuda'

In [ ]:
training_args = TrainingArguments("trained_scibert_ner_model", # output dir
                                      learning_rate=2.768901750544875e-05,
                                      weight_decay=0.0273138487143614,
                                      num_train_epochs=8,
                                      dataloader_drop_last=True,
                                      per_device_eval_batch_size=4,
                                      per_device_train_batch_size=4,
                                      logging_steps=50,
                                      save_steps=len(train) // batch_size,
                                      lr_scheduler_type='cosine',
                                      evaluation_strategy='steps',
                                      eval_steps=len(train) // batch_size,
                                      report_to='all'
                                      )

#learning_rate 1.5441110254159498e-05
#num_train_epochs 12
#weight_decay 0.21467682833420043
#per_device_train_batch_size 16

#load_param()

In [ ]:
loop_val = 4
loop_results = np.zeros(loop_val)
loop_resultss = np.zeros(loop_val)
for r in range(loop_val):

  ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training

  ner_model.sci_embeddings.requires_grad = False

  trainer = MultilabelTrainer(
      model=ner_model,
      args=training_args,
      train_dataset=train,
      eval_dataset=test,
      data_collator=collator  # defines how to merge data into batches, using the collator function above
  )

  # Print
  print(f'Train run #{r}')
  print('--------------------------------')

  trainer.train()

  # Process is complete.
  print('Training process has finished.')

  # Print about testing
  print('Starting testing')

  with torch.no_grad():
    result = trainer.evaluate(test)
    print(result)

    # Print accuracy
    print('Accuracy for fold ', r, ': ', result['eval_micro_f1_no_o'], ' -- ', result['eval_micro_f1'])
    print('--------------------------------')
    loop_results[r] = result['eval_micro_f1_no_o']
    loop_resultss[r] = result['eval_micro_f1']
    del result

  if r > 0:
    if loop_results[r] < loop_results[r-1]:
      save_path = f'./model-fold-{r}.pth'
      torch.save(ner_model.state_dict(), save_path)

  print('Testing process has finished.')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8816
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 17632


Train run #0
--------------------------------


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
2204,0.016100,0.054301,0.581655,0.499412,0.724854,0.886183,0.886183,0.886183,0.618525,0.519438,0.767768,0.620675,0.514202,0.782758


Saving model checkpoint to trained_scibert_ner_model/checkpoint-2204
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
2204,0.016100,0.054301,0.581655,0.499412,0.724854,0.886183,0.886183,0.886183,0.618525,0.519438,0.767768,0.620675,0.514202,0.782758
4408,0.003200,0.061881,0.611389,0.535692,0.739636,0.900365,0.900365,0.900365,0.653316,0.565506,0.777773,0.645373,0.551505,0.777748
6612,0.011300,0.081936,0.642563,0.607479,0.691803,0.918235,0.918235,0.918235,0.668394,0.636288,0.716518,0.647776,0.578871,0.735302
8816,0.000500,0.086232,0.634516,0.580115,0.718082,0.915955,0.915955,0.915955,0.673809,0.616821,0.749274,0.665255,0.600722,0.745320
11020,0.000100,0.080817,0.659047,0.604527,0.736668,0.922052,0.922052,0.922052,0.680355,0.603674,0.782386,0.673119,0.592385,0.779330
13224,0.000400,0.082161,0.657853,0.591638,0.759868,0.913772,0.913772,0.913772,0.688081,0.611719,0.790673,0.677800,0.595528,0.786449
15428,0.000000,0.103284,0.671359,0.624551,0.734502,0.926564,0.926564,0.926564,0.701223,0.654301,0.758283,0.693414,0.640116,0.756393
17632,0.000000,0.104851,0.673620,0.628488,0.735008,0.927421,0.927421,0.927421,0.703511,0.656722,0.760267,0.695747,0.642204,0.759030


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8816
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 17632


Testing process has finished.
Train run #3
--------------------------------


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
2204,0.000100,0.172189,0.650070,0.615769,0.710723,0.922278,0.922278,0.922278,0.666230,0.657918,0.684872,0.663887,0.620587,0.713683
4408,0.000100,0.131400,0.653734,0.580293,0.772156,0.910327,0.910327,0.910327,0.671835,0.591412,0.782492,0.664122,0.578299,0.779858
6612,0.000100,0.122860,0.664354,0.611580,0.735644,0.921745,0.921745,0.921745,0.681167,0.630130,0.750801,0.678220,0.635275,0.727393
8816,0.000100,0.119737,0.634852,0.551171,0.781672,0.906689,0.906689,0.906689,0.644965,0.554492,0.780912,0.642323,0.557148,0.758239
11020,0.000000,0.139423,0.675162,0.634404,0.729996,0.925998,0.925998,0.925998,0.690435,0.652579,0.743762,0.679919,0.622861,0.748484
13224,0.000000,0.145027,0.677469,0.631677,0.737263,0.928084,0.928084,0.928084,0.693349,0.634243,0.770373,0.687375,0.620697,0.770103
15428,0.000000,0.148289,0.679412,0.634744,0.735759,0.929200,0.929200,0.929200,0.700800,0.655038,0.756863,0.697921,0.654361,0.747693
17632,0.000000,0.150659,0.678142,0.631840,0.737151,0.929022,0.929022,0.929022,0.698540,0.649705,0.758897,0.696014,0.649031,0.750330


Saving model checkpoint to trained_scibert_ner_model/checkpoint-2204
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-4408
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-6612
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-8816
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-11020
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-13224
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-15428
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint 

Training process has finished.
Starting testing
{'eval_macro_f1': 0.6781416903871643, 'eval_macro_precision': 0.6318404591442596, 'eval_macro_recall': 0.7371512651503895, 'eval_micro_f1': 0.9290219289734135, 'eval_micro_precision': 0.9290219289734135, 'eval_micro_recall': 0.9290219289734135, 'eval_macro_f1_no_o': 0.6985402448838623, 'eval_macro_precision_no_o': 0.64970505711993, 'eval_macro_recall_no_o': 0.758897329001687, 'eval_micro_f1_no_o': 0.6960136952800196, 'eval_micro_precision_no_o': 0.6490307867730901, 'eval_micro_recall_no_o': 0.7503295544423939, 'eval_loss': 0.15065874749874683, 'eval_runtime': 42.8803, 'eval_samples_per_second': 11.52, 'eval_steps_per_second': 2.892, 'epoch': 8.0}
Accuracy for fold  3 :  0.6960136952800196  --  0.9290219289734135
--------------------------------
Testing process has finished.


In [ ]:
loop_results

array([0.69574674, 0.69016833, 0.68987113, 0.6960137 ])

In [ ]:
loop_resultss

array([0.92742092, 0.92902193, 0.92898959, 0.92902193])

In [ ]:
sum = 0.0
for value in loop_results:
  sum += value
print(f'Average micro_f1_no_o: {sum/len(loop_results)} %')

sum = 0.0
for value in loop_resultss:
  sum += value
print(f'Average micro_f1: {sum/len(loop_results)} %')

Average micro_f1_no_o: 0.6929499743727167 %
Average micro_f1: 0.9286135907885373 %


In [ ]:
ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training
ner_model.load_state_dict(torch.load(save_path))
ner_model.eval()

NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
A=100
print(A)

100


# Get Values of Thruth

In [ ]:
# Pytorch thing (if we aren't training, do this)
ner_model.eval()
ner_model.to('cuda')

NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
output_preds = []
output_real = []
for x in range(len(test)):
  inputs1 = test[x]['input_ids'][0].clone().detach().to(torch.long).unsqueeze(0).to('cuda')
  inputs2 = test[x]['attention_mask'][0].clone().detach().to(torch.long).unsqueeze(0).to('cuda')
  inputs = {'input_ids': inputs1,  'attention_mask': inputs2}
  #print(inputs)

  temp_test = test[x]
  temp_out = temp_test.pop("labels")
  output_real.append(np.array(temp_out[temp_out != -100]))

  gen_preds = ner_model(**inputs)
  label_preds = np.argmax(gen_preds.cpu().detach().numpy(), axis=-1)[0]
  output_preds.append(label_preds[temp_out != -100])


In [ ]:
for x in range(len(output_real)):
  output_real[x] = [ID2Entity(y) for y in output_real[x]]
  output_preds[x] = [ID2Entity(z) for z in output_preds[x]]

In [ ]:
#output_real[28]

In [ ]:
from sklearn.metrics import mean_squared_error, multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15, 12]
plt.rcParams["figure.autolayout"] = True
plt.rcParams.update({'font.size': 13})

labels = ["O", "B-Task", "I-Task", "B-Method", "I-Method", "B-Metric", "I-Metric", "B-Material", "I-Material", "B-OtherScientificTerm", "I-OtherScientificTerm", "B-Generic", "I-Generic"]
cm = confusion_matrix(output_real[0], output_preds[0], labels=labels)
for x in range(len(output_real)-1):
  cm += confusion_matrix(output_real[x+1], output_preds[x+1], labels=labels)
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)

In [ ]:
del cm

In [ ]:
len(output_real)

# Test over Text

In [ ]:
def prepare_input(txt):
  inputs = BertTokenizer(txt, return_tensors='pt', padding='max_length', truncation=True, max_length=150).to('cuda')
  return inputs

input_text = ["English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement."]
##input_text = ["The transient analysis of gyro-elastic structured media, composed of periodically placed masses interconnected by elastic rods and attached to gyroscopic spinners, is presented."]
##input_text = ["The results indicated that thermal curing promoted the early strength of mortars, while decreased the late strength of mortars."]
##input_text = ["A wide variety of processes are attested in the literature, and we find different forms of clippings in our data, including mixtures of different clippings, homophone respellings, phonetic respellings in-cluding informal oral forms, initialisms (but no acronyms), and mixtures of clipping together with homo-phone and phonetic respellings."]

#input_text = ["The goal is to accurately predict the running time of applications for task scheduling and job migration."]
#input_text = ["This paper reports on the development of a cross-domain framework for describing complex design practices."]
#input_text = ["Studies of inequality in China typically ignore cost of liv-ing differences between areas."]
#input_text = ["The present study was designed to explore the long-term differences be-tween three mouse models for depression."]
#
input_text = ["Finally, regarding professional competencies, teachers appeared to be largely unprepared to conduct language assessments consistent with the LAR demands."]

##input_text = ["propose a fast and reliable restoration method of virtual resources on OpenStack when physical servers or virtual machines are down."]
##input_text = ["The results from our simulations reveal that the network assisted adaptation clearly outperforms the purely client-based DASH heuristics in some of the metrics, not all of them, particularly, in situations when the achievable throughput is moderately high or the link quality of the mobile clients does not differ from each other substantially."]
##input_text = ["For hard rock drilling in coal mine, the drilling efficiency and service life of polycrystalline diamond compact bit are very low."]
##input_text = ["Capturing changes in foreign reserves and exchange rates through the exchange market pressure, this article investigates whether economic policy uncertainty plays any role in exchange market pressure movements while controlling for the effects of domestic and external factors."]
##input_text = ["This paper presents design of an self contained actuators unit in wide area damping control of power system in stabilizing system response for both nominal system condition and during actuator faults."]

##input_text = ["Ultrasound-based brain stimulation techniques may become a powerful new technique to modulate the human brain in a focal and targeted manner."]
#input_text = ["Recent work pre-training Transformers with self-supervised objectives on large text corpora has shown great success when fine-tuned on downstream NLP tasks including text summarization."]

# Tokenize + pad
inputs = prepare_input(input_text)

#inputs
#print(inputs)

In [ ]:
# Pytorch thing (if we aren't training, do this)
ner_model.eval()

# Get predictions
preds = ner_model(**inputs).cpu().detach().numpy()
preds = np.argmax(preds, axis=-1)[0]
pred_labels = [ID2Entity(x) for x in preds]

# Convert token ids to text
tokens = BertTokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Display result
for token, label in zip(tokens, pred_labels):
  if token == '[SEP]':
    break
  if token == '[CLS]':
    continue
  print('{} -> {}'.format(token, label))

finally -> None
, -> None
regarding -> None
professional -> None
competencies -> None
, -> None
teachers -> None
appeared -> None
to -> None
be -> None
largely -> None
un -> None
##prep -> None
##ared -> None
to -> None
conduct -> None
language -> None
assessments -> None
consistent -> None
with -> None
the -> None
lar -> B
demands -> None
. -> None


# Model Save and Load

In [ ]:
print("Our model: \n\n", ner_model, '\n')
print("The state dict keys: \n\n", ner_model.state_dict().keys())

Our model: 

 NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
torch.save(ner_model.state_dict(), 'trained_model_dic.pth')

In [ ]:
# download checkpoint file
files.download('trained_model_dic.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls

checkpoint.pth	  model-fold-1.pth  model-fold-4.pth  trained_model_dic.pth
dev.json	  model-fold-2.pth  sample_data       trained_scibert_ner_model
model-fold-0.pth  model-fold-3.pth  test.json	      train.json


Loading the model

In [ ]:
!ls

sample_data  test_500_v2.conll	train_1500_v2.conll  trained_model_dic.pth


In [ ]:
state_dict = torch.load('trained_model_dic.pth')
print(state_dict.keys())

odict_keys(['sci_embeddings.embeddings.position_ids', 'sci_embeddings.embeddings.word_embeddings.weight', 'sci_embeddings.embeddings.position_embeddings.weight', 'sci_embeddings.embeddings.token_type_embeddings.weight', 'sci_embeddings.embeddings.LayerNorm.weight', 'sci_embeddings.embeddings.LayerNorm.bias', 'sci_embeddings.encoder.layer.0.attention.self.query.weight', 'sci_embeddings.encoder.layer.0.attention.self.query.bias', 'sci_embeddings.encoder.layer.0.attention.self.key.weight', 'sci_embeddings.encoder.layer.0.attention.self.key.bias', 'sci_embeddings.encoder.layer.0.attention.self.value.weight', 'sci_embeddings.encoder.layer.0.attention.self.value.bias', 'sci_embeddings.encoder.layer.0.attention.output.dense.weight', 'sci_embeddings.encoder.layer.0.attention.output.dense.bias', 'sci_embeddings.encoder.layer.0.attention.output.LayerNorm.weight', 'sci_embeddings.encoder.layer.0.attention.output.LayerNorm.bias', 'sci_embeddings.encoder.layer.0.intermediate.dense.weight', 'sci_emb

In [ ]:
ner_model = NerModel(BertEmbModel).to('cuda')
ner_model.load_state_dict(state_dict)

<All keys matched successfully>

# Obtain datasets' weights values (Do not run - fixed values)

In [ ]:
zero = 0
one=0
two=0
three=0
four=0
five=0
six=0
local_set = train
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1

print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)

KeyboardInterrupt: ignored

In [ ]:
zero = 0
one=0
two=0
three=0
four=0
five=0
six=0
local_set = test
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1

print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)

In [ ]:
zero = 0
one=0
two=0
three=0
four=0
five=0
six=0
local_set = val
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1

print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)